In [ ]:
import cv2
import numpy as np
import os
from concurrent.futures import ThreadPoolExecutor
import time
from PIL import Image
from pdf2image import convert_from_path
from pdf2image.generators import threadsafe

In [ ]:
begin = time.time()

input_dir = 'input'
output_dir = 'output'
gamma = 2.5
filename_replace = 'file'
filename_replace__v = ''
extension = 'jpg'
pdf_input = f'pdf/input.pdf'
pdf_output = f'pdf/output.pdf'

In [ ]:
# clean input and output folder
for file in os.scandir(input_dir):
  os.remove(file.path)
for file in os.scandir(output_dir):
  os.remove(file.path)

In [ ]:
@threadsafe
def output_file_generator():
  i = 0
  while True:
    i += 1
    yield str(filename_replace)

In [ ]:
files = convert_from_path(
    pdf_input, output_folder=input_dir, fmt=extension, dpi=100,
    thread_count=5, output_file=output_file_generator(), grayscale=True
)
for file in files:
    file.close()

In [ ]:
def optimize(gamma: float, filename: str, log: bool=False):
  """伽马变换"""
  if log:
    print(f'process file by gamma {gamma}: {filename}')
  image = cv2.imread(f'{input_dir}/{filename}', cv2.IMREAD_GRAYSCALE)
  fI = image / 255.0
  O = np.power(fI, gamma)
  # need to extent to unit8
  O = O * 255
  filename = f"{output_dir}/{filename}"
  cv2.imwrite(filename, O)

In [ ]:
b = time.time()
pool = ThreadPoolExecutor(5)
for file in os.scandir(input_dir):
  if file.is_file():
    pool.submit(optimize, gamma, file.name)
pool.shutdown(wait=True)
print(f'time cost: {time.time() - b}s')

In [ ]:
b = time.time()
images = []
for file in os.listdir(output_dir):
  images.append(f'{output_dir}/{file}')
images.sort(key=lambda x: int(x.replace(f'.{extension}', '').replace(f'{output_dir}/{filename_replace}-', '')))
output = []
for img in images:
  output.append(Image.open(img))
if len(output) > 0:
  first_img = Image.open(images[0])
  output.pop(0)
  first_img.save(pdf_output, 'pdf', save_all=True, append_images=output)
print(f'time cost: {time.time() - b}s')

In [ ]:
print(f'total time cost: {time.time() - begin}s')